In [7]:
!pip install -q bs4

In [2]:
def get_market_info(market_file_path):
    with open(market_file_path) as cookie_file:
        def get_valuepair_from_line(line):
            idx = line.find(' ')
            key = line[0:idx].strip()
            value = line[idx:].strip()
            return [key,value]

        pairs = [get_valuepair_from_line(line) for line in cookie_file.readlines()]
        file_info = {key_value[0]: key_value[1] for key_value in pairs}

        print(f"Loaded market with url '{file_info['Url']}' and cookie '{file_info['Cookie']}'")
        return file_info

In [54]:
from requests_tor import RequestsTor
from bs4 import BeautifulSoup

def get_page_list(url, cookie):
    headers = {
        "Cookie": cookie,
    }

    html = rt.get(url, headers=headers).text
    soup = BeautifulSoup(html)
    pagination_elements = soup.find_all('a',{'class': 'pagination_link'})
    if len(pagination_elements) < 1:
        print('No items returned, please check if session is not expired')
        print(str(soup))
        return []

    href_lastpage = pagination_elements[-1].get('href')
    last_page_nr = int(href_lastpage.split('pg=')[1])

    urls_to_scrape = [f"{url}?category=0&amp;pg={page_nr}" for page_nr in range(1,last_page_nr+1)]

    return urls_to_scrape

def scrape_page(url, cookie):
    headers = {
        "Cookie": cookie,
    }

    html = rt.get(url, headers=headers).text
    soup = BeautifulSoup(html)
    return soup

def get_products_from_page(soup):
    products = soup.find_all('div', {'class':'wLf'})
    print(products)
    return products

In [49]:
rt = RequestsTor()
market_info = get_market_info('./marketinfo_vice.txt')

urls_to_scrape = get_page_list(market_info['Url'], market_info['Cookie'])

Loaded market with url 'http://52qlucglu6fuaqist2herssakipapig2higaaayu7446n55xw4ylxqid.onion' and cookie 'loomer323232=ad53a936897cc2f3a8f6156d60dc47f9; vicecityprotect=5edc3bed517dbf9865fb815552c1eb21; prtector25edc3bed517dbf9865fb815552c1eb2160=lol; cookiejar=2I5J6ED6LK5X2QLZDOMDVTIFTXW00Y; dcap=60C2435026F12898F37ED4B41F9F5371805419235932865843DBC7CA18E054884402B06BCCDD24FCEA9CEA50182EEA50; PHPSESSID=na9b4oo04utb16a9n17nsp9kc0'


In [57]:
def extract_id_from_product(product):
    href = product.find('div',{'class':'wLfLeft'}).find('a').get('href')
    split = href.split('lid=')
    return split[1]

for url_to_scrape in urls_to_scrape:
    print()
    print('Scraping page:', url_to_scrape)

    page = scrape_page(url_to_scrape, market_info['Cookie'])
    products = get_products_from_page(page)

    if len(products) == 0:
        print('There are no products on url:', url_to_scrape, 'maybe the session has expired?')
        print('logging page html:')
        print(str(page))
        print()
        continue
    for product in products:
        product_id = extract_id_from_product(product)
        print('Product with id:', product_id)

        with open(f'./vice_products/product_{product_id}.html','w') as product_file:
            html = product.prettify()
            product_file.write(html)




[<div class="wLf"><div class="wLfLeft"><div class="wLfMultisigPayment"></div><div class="wLfPaymentType"><img src="data:png;base64,iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAYAAABzenr0AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAAJOgAACToB8GSSSgAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAANNSURBVFiF7ZfPbxtFFMc/M3biRMUKEFPJwZFAFaD0AALa2LgSIpW40EtBIkKkAeXGnQPHCnGi4oCExB8ATdu0peFCDlwiuCSLQqFc0uBEBmrsNk1oTJO2650fHILdrL12/CMcELzTzJs38/nu27czs/BfN9HOpNnV2QciKpoIY/YDKOSqG76dG9k/svmPCZi/Nv8EUp5A2FeA5wPmGgvfY5mxRp9OD6aX90SAk3P6CXHSWt4BuprUWwLxqdHqg/Rg+o+2BTjXnaPWcB7obxJcbWsIO5qKp2brBch6A3OF70YxzHQAB4hhxYxTcF6vFxCYgbnC3EvCyq9pPuW7mWeEeDkdH/5mVwHfFi4/0m29K0B8j+B/k0ReaPt0MpFc3+mueQUR672/F3BlNPeUe99h7QAhTtbo2tlxrjuPW8NVoLsTuKc9zi5N4WqXE0Nj9IZ7yjBXoZ46MnDk13KsLwPW2IlO4doqvlie5rfb17hxZ5WvsjP314eItOGJnfF+AYhXO4GXTIkzi1Msb6wAEO2KcnRwxBcjBMd9/XJjIb8QU+ib7cKVUUwtXST7Z7YCHz84xsM9D1WHWqN1rLxBVTJgjEl0BP/5QjNwACGRFVZFgA6JupWf28xxaXkabVXN2PY7v0S2+AsA+7r2MTb0Rj04ACYkB8rtcIMHAyC/mefs1fO42sXVHqNPvkZIhCvwi5lpMrdWKvDxoTeJ9cZ2W